# **Practical 9**

# **Sentiment Analysis of Social Media**

In [1]:
!pip install sklearn

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Jay S
[nltk_data]     Khatri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [4]:
df=pd.read_csv("Dataset/train.csv")

In [5]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [6]:
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [7]:
df.tweet=df.tweet.apply(lambda x:re.sub(r"[^A-Za-z]"," ",x))

In [8]:
def clean_data(row):
    tokens=row.split()
    word_list=[]
    for x in tokens:
        x=porter.stem(x.lower())
        if (x not in stop_words) and len(x)>2:
            word_list.append(x.lower())
    return (" ".join(word_list))   

In [9]:
df['clean_tweet']=df.tweet.apply(lambda x:clean_data(x))

In [10]:
df.head()

,id,label,tweet,clean_tweet
0,1,0,fingerprint Pregnancy Test https goo gl h ...,fingerprint pregnanc test http goo mfqv androi...
1,2,0,Finally a transparant silicon case Thanks t...,final transpar silicon case thank uncl yay son...
2,3,0,We love this Would you go talk makememorie...,love thi would talk makememori unplug relax ip...
3,4,0,I m wired I know I m George I was made that wa...,wire know georg made way iphon cute daventri h...
4,5,1,What amazing service Apple won t even talk to...,amaz servic appl even talk question unless pay...


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_tweet'], 
    df['label'], 
    random_state = 1
)

In [12]:
print("size of training dataset:{0}\nsize of testing dataset:{1}".format(len(X_train),len(X_test)))

size of training dataset:5940
size of testing dataset:1980


In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(y_train)
Test_Y = Encoder.fit_transform(y_test)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['clean_tweet'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [15]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
import sklearn.svm as svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  87.67676767676768
